# Analizando la corrupción en el congreso

Probablemente a éstas alturas a nadie se le escapa que en España hay un problema bastante grande en cuanto a la corrupción. Vamos por lo menos a sacarle un poco de partido haciendo un poco de estadística básica con Python, lo que nos permitirá afianzar algunos conceptos.

## Introduciendo los diccionarios

Los diccionarios son en realidad muy similares a las listas, con la diferencia de que el índice de cada elemento puede no ser un entero ordenado de 0 al número de elementos menos 1. Veamos un ejemplo de como podemos crear un diccionario con datos relevantes del partido político PP:

In [294]:
PP = {'nombre':'Partido Popular',
      'afiliados':(865000, 100000),
      'escanos':119,
      'imputados':835}

Fuentes:

* https://es.wikipedia.org/wiki/Partido_Popular
* https://www.elplural.com/2016/05/30/los-31-casos-con-sus-835-imputados-que-acorralan-al-pp

Por lo visto, el número de militantes del PP no es un número muy fácil de obtener, ya que por lo visto cuentan entre sus filas con militantes ya defuntos. Por tanto, vamos a mantener tanto el caso más favorable para el partido (su propio "censo") como el más desfavorable (datos provistos por un analista independiente).

Ya se puede intuir que los diccionarios se pueden asociar a listas con referencias más inteligibles. Por ejemplo, si quisieramos conocer el nombre del partido podríamos solicitarlo de la siguiente manera:

In [295]:
print(PP['nombre'])

Partido Popular


Además, podemos manipular los diccionarios con cierta soltura. Por ejemplo, podemos añadirles nuevos atributos simplemente especificando la nueva entrada:

In [296]:
PP['acronimo'] = 'PP'
PP

{'acronimo': 'PP',
 'afiliados': (865000, 100000),
 'escanos': 119,
 'imputados': 835,
 'nombre': 'Partido Popular'}

Ahora que ya conocemos lo básico sobre los diccionarios, vamos a crear otros partidos políticos.

In [297]:
PSOE = {'nombre':'Partido Socialista Obrero Español',
        'acronimo':'PSOE',
        'afiliados':(187360,),
        'escanos':89,
        'imputados':320}
PODEMOS = {'nombre':'Podemos',
      'acronimo':'Podemos',
      'afiliados':(489094,),
      'escanos':65,
      'imputados':13}
CS = {'nombre':'Ciudadanos',
      'acronimo':'Cs',
      'afiliados':(20300,),
      'escanos':40,
      'imputados':15}

Fuentes:

* https://es.wikipedia.org/wiki/Partido_Socialista_Obrero_Espa%C3%B1ol
* https://www.infolibre.es/noticias/politica/2016/02/26/el_psoe_tiene_320_cargos_excargos_investigados_imputados_45590_1012.html
* https://politica.elpais.com/politica/2017/04/18/actualidad/1492533676_586452.html
* https://www.vozpopuli.com/espana/Union-_Progreso_y_Democracia_-UPyD-Ciudadanos-Albert_Rivera-Militantes-UPyD_0_793420652.html
* https://movimientoindignadosspanishrevolution.wordpress.com/2015/05/23/los-imputados-en-ciudadanos/
* https://participa.podemos.info/es
* https://www.esdiario.com/468903306/La-larga-lista-de-cargos-publicos-imputados-de-Podemos-que-oculta-Pablo-Iglesias.html

Bien es cierto que en la lista de Podemos varios imputados poco o nada tienen que ver con la corrupción. Pero creo que es efectivamente más riguroso mantenerlos dentro del cálculo. Respecto de Ciudadanos, en realidad desconozco en absoluto las causas de las imputaciones (y de hecho no las he investigado).

## La función principal, main()

Como en el caso anterior, vamos a comenzar por la función principal. Dicha función tan sólo va a recorrer los partidos, y va a pedir el número de corruptos que hay en el congreso de cada uno de ellos, con un nivel de significación determinado.

In [298]:
def main(partidos=(PP, PSOE, PODEMOS, CS), certeza=0.05):
    print('Una probabilidad del {:.1f}% de que...'.format(
        certeza * 100))
    for partido in partidos:
        corruptos = escanos_podridos(partido['escanos'],
                                     partido['imputados'],
                                     partido['afiliados'],
                                     certeza)
        print('Hay AL MENOS entre {} y {} "manzanas podridas" sentadas en la bancada de {} del congreso'.format(
            corruptos[0], corruptos[1], partido['acronimo']))

Se puede apreciar que la función main tiene algunos parámetros con valores por defecto, lo que quiere decir que, si no especificamos los *partidos*, entonces se usarán PP, PSOE, PODEMOS y CS, y si no especificamos la *certeza*, entonces se calculará con un nivel de significación del 5%.

En la función anterior sólo necesitamos implementar la función *escanos_podridos*.

## Calculando el número de manzanas podridas de un partido

Como en cualquier otro problema, los resultados derivan directamente de las hipótesis practicadas. En éste caso el razonamiento va a ser el siguiente:

Primero de todo, tomaremos como población base el número de afiliados, lo que quiere decir que nuestra probabilidad de encontrar a un corrupto será el número de imputados dividido por el número de afiliados. Bien es cierto que el número de imputados puede diferir finalmente del número de corruptos, dependiendo de la sentencia final. Dicho esto, una imputación (ahora se le denomina calidad de investigado) implica que hay indicios claros de que la persona está envuelta en el caso que se investiga.

Por otro lado, también es cierto que admitir que todos los afiliados son potenciales políticos corruptos es una tesis muy conservadora, pues la inmensa mayoría de ellos nunca alcanzarán ningún tipo de responsabilidad política que les de la oportunidad de corromperse.

Así pues, creo que se podría asegurar que las hipótesis practicadas están claramente del lado de la presunción de inocencia.

Una vez tengamos la probabilidad base, que es la probabilidad de que elegido un militante al azar del partido, demos con un corrupto, calcularemos la probabilidad de que haya 0 corruptos de entre todos los miembros de la bancada, 1 corrupto, 2 corruptos, y así consecutivamente hasta calcular la probabilidad de que todos los congresistas del partido sean "manzanas podridas".

Ese datos nos permitirá calcular el histograma, que es en realidad la probabilidad de que un determinado número de congresistas (**o más**) sean corruptos.

Por último buscaremos en el histograma el punto en el que la probabilidad es mayor que la *certeza* solicitada, y tendremos el número de "manzanas podridas" que buscabamos.

¡Veámoslo implementado!

In [299]:
def escanos_podridos(escanos, imputados, afiliados, certeza):
    return (manzanas_podridas(escanos, imputados, max(afiliados), certeza),
            manzanas_podridas(escanos, imputados, min(afiliados), certeza))

Bueno, en realidad ésta función no es muy emocionante, tan sólo manda el trabajo a una segunda función, *manzanas_podridas*, para calcular el número de afectados en el mejor y el peor de los casos. Vamos por tanto a ver la función *manzanas_podridas*.

In [300]:
def manzanas_podridas(escanos, imputados, afiliados, certeza):
    p = imputados / afiliados  # Probabilidad base de encontrar un corrupto
    # Ahora vamos a ir calculando la probabilidad de que haya EXACTAMENTE
    # 0 corruptos en el congreso, 1 corrupto, 2 corruptos...
    probs = []
    for n in range(escanos + 1):  # +1 para que el bucle incluya n=0 y n=escanos
        probs.append(probabilidad(p, n, escanos))
    # Ahora calculamos el histograma. Como en realidad nos interesa la
    # probabilidad de que haya N, O MAS, corruptos, vamos a calcular el
    # histograma de Probs, pero dado la vuelta
    h = histograma(probs[::-1])
    # Y por fin, vamos a recorrer el histograma para encontrar el punto en
    # el que tenemos certeza suficiente del numero de corruptos
    n = 0
    while h[n] < certeza:
        n += 1
    return escanos - n

La función anterior requiere la implementación de dos funciones, *probabilidad* e *histograma*. Parece obvio que la función histograma es más sencilla, así que vayamos con ella.

In [301]:
def histograma(probabilidades):
    # Vamos a añadir un cero al principio del histograma, por conveniencia.
    # Seria algo asi como la probabilidad de que haya mas congresistas
    # corruptos que congresistas. Ya lo eliminaremos al final
    h = [0]
    for p in probabilidades:
        h.append(p + h[-1])
    # Devolvemos el histograma, quitando el cero del principio
    return h[1:]        

Y ahora vamos a calcular la probabilidad de que haya *n* congresistas corruptos en el congreso, lo que es básicamente es un problema de combinatoria. Para ello, vamos a volver al concepto de probabilidad base, *P*. Como ya hemos mencionado, *P* es la probabilidad de que un militante al azar resulte ser un político corrupto, y por tanto, la probabilidad de que un militante al azar sea honrado, *H*, la podemos calcular como:

$H = 1 - P$.

Aquí conviene hacer una aclaración. Realmente, al extraer un militante del grupo, dependiendo de si es corrupto o no, las probabilidades de que el siguiente sea corrupto (la probabilidad base) varía, pues el número de militantes, y eventualmente de imputados, varía. No obstante, siendo el número de militantes grande comparado con el de escaños, podemos ignorar este efecto, que simplificará enormemente el problema.

Ahora ciñamonós a un ejemplo de un partido con sólo 3 escaños. Si queremos calcular la probabilidad de que todos ellos sean honrados sería sencillamente:

$P_{0,3} = H^3$.

De la misma forma, la probabilidad de que todos ellos sean corruptos se puede calcular como:

$P_{3,3} = P^3$.

Pero si deseamos saber la probabilidad de que uno de ellos sea un corrupto, la cosa se complica... Ya que cabe la posibilidad de que sea el primero el que es un corrupto, pero también cabe la posibilidad de que sea el segundo, o el tercero, o el cuarto:

$P_{1,3} = P \cdot H \cdot H + H \cdot P \cdot H + H \cdot H \cdot P = 3 \cdot P \cdot H^2$.

De la misma forma, si buscamos la probabilidad de que haya 2 corruptos, podemos figurarnos que sean el primero y el segundo, o el primero y el tercero, o el segundo y el tercero:

$P_{2,3} = P \cdot P \cdot H + P \cdot H \cdot P + H \cdot P \cdot P = 3 \cdot P^2 \cdot H$.

Generalizando un poco, la probabilidad de encontrar $n$ corruptos entre $N$ congresistas se puede calcular como

$P_{n,N} = C_n^N \cdot P^n \cdot H^{N - n}$,

Con

$C_n^N = \frac{N!}{n! \cdot (N - n)!}$

Ver https://es.wikipedia.org/wiki/Combinatoria

¡Así que procedamos a implementarlo!

In [302]:
def probabilidad(p, n, N):
    c = factorial(N) / (factorial(n) * factorial(N - n))
    h = 1 - p
    return c * p**n * h**(N - n)

Evidentemente nos hace falta una función para calcular el factorial. Éste puede ser un asunto controvertido, así que lo mejor es usar directamente la función *factorial* que nos provee Python en la librería matemática:

In [303]:
from math import factorial

## Resultado

Bueno, vamos a ver que nos predice nuestro programita...

In [304]:
main()

Una probabilidad del 5.0% de que...
Hay AL MENOS entre 1 y 3 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Cs del congreso


Dicho de otra forma, no es estadísticamente irracional sugerir que en el congreso aún puede haber "manzanas podridas" por descubrir (sólo en esos 4 partidos políticos). De hecho jugando un poco podemos descubrir la probabilidad de que haya congresistas corruptos...

In [305]:
main(certeza=0.141)

Una probabilidad del 14.1% de que...
Hay AL MENOS entre 0 y 2 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Cs del congreso


Es decir, que hay un **14.1% de probabilidades de que en la bancada del PSOE terminen imputando a alguien**.

Respecto de la bancada del PP, pues depende del número de afiliados que consideremos...

In [306]:
main(certeza=0.108)
main(certeza=0.631)

Una probabilidad del 10.8% de que...
Hay AL MENOS entre 1 y 2 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Cs del congreso
Una probabilidad del 63.1% de que...
Hay AL MENOS entre 0 y 1 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Cs del congreso


Pero sea como sea, **hay una probabilidad de entre el 10.8% y el 63.1% de que alguno de los congresistas del PP acabe siendo juzgado**. A título personal, tiendo a creer más en el número de afiliados reportados por una entidad independiente al número de afiliados calculados por el propio PP, donde se han demostrado irregularidades. Sea como sea, aún en el mejor de los casos, una probabilidad de casi el 11% no es nada desdeñable.

¿Y qué pasa con Podemos y Ciudadanos?

In [307]:
main(certeza=0.029)
main(certeza=0.001)

Una probabilidad del 2.9% de que...
Hay AL MENOS entre 1 y 3 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de Cs del congreso
Una probabilidad del 0.1% de que...
Hay AL MENOS entre 2 y 5 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 2 y 2 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de Cs del congreso


Pues resulta que **Ciudadanos tiene una probabilidad del 2.9% de tener algún corrupto en el congreso**, mientras que **Podemos tiene una probabilidad del 0.1% de que algún congresista acabe siendo imputado**.

## Conclusiones Pythonicas

En éste ejemplo hemos vuelto a tratar un problema moderadamente sencillo haciendo mun uso extensivo de funciones, siguiendo siempre la máxima de no construir la casa por el tejado. Hemos comenzado construyendo nuestra función principal, cuyo objetivo no era otro que analizar el eventual número de congresistas corruptos, dada una determinada probabilidad. Una vez hemos contruido dicha función, la hemos analizado para ver que dependencias/funciones no estaban implementadas, y nos hemos puesto a ello.

Esa forma de pensar simplifica los razonamientos, pues en todo momento se tiene claro el objetivo, invita de forma natural a usar funciones, ya que de alguna forma te pide que el trabajo complicado lo dejes para una función a implementar más adelante, y minimiza las constantes idas y venidas en el código, lo que puede evitar que en determinado momento te pierdas y necesites repasar para volver al hilo de tu razonamiento inicial.

## Conclusiones políticas

¿Qué decir de los números? Estamos diciendo que en una de las instituciones más importantes de nuestro país, donde se supone que se deben situar personas de una honestidad incuestionable, tenemos probabilidades por encima del 10% de poder encontrar políticos envueltos en casos de corrupción.

La cosa se vuelve aún más peliaguda en el caso del Partido Popular, puesto que si aceptamos que el número de afiliados se aproxima más a 100000 (número alcanzado por una entidad independiente) que al casi millón que asegura el PP, muertos inclusive, entonces tenemos más de un 50% de probabilidades de que alguno de los congresistas esté aún envuelto en algún caso de corrupción, y es estadísticamente razonable sugerir que puede haber 3 o más congresistas corruptos en su bancada. Por supuesto, no sería el primer miembro del congreso, por parte del PP, que se ve envuelto en algún caso de corrupción.

Además dichos números son bastante conservadores, pues utilizan únicamente el número de afiliados, y no el número real de personas de cada partido con capacidad real de ser corruptos, número éste último que probablemente sea significativamente menor. Para dar algunos números, se ha llegado a afirmar que hay 557516 cargos públicos en España (https://verdaderaizquierda.blogspot.com.es/2012/06/numero-de-cargos-publicos-en-espana.html), sin embargo parece que unos 125000 cargos públicos se acerca mucho más a la cifra real (http://www.eldiario.es/agendapublica/nueva-politica/politicos-Espana-importa-saberlo_0_230277794.html). Si por experimentar un poco asumimos que los cargos públicos se reparten de forma proporcional a los escaños:

In [308]:
c_publicos = 125000
for partido in (PP, PSOE, PODEMOS, CS):
    partido['afiliados'] = (c_publicos * partido['escanos'] / 350,)

In [349]:
main(certeza=0.905)
main(certeza=0.593)
main(certeza=0.041)
main(certeza=0.035)

Una probabilidad del 90.5% de que...
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Cs del congreso
Una probabilidad del 59.3% de que...
Hay AL MENOS entre 2 y 2 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 1 y 1 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Podemos del congreso
Hay AL MENOS entre 0 y 0 "manzanas podridas" sentadas en la bancada de Cs del congreso
Una probabilidad del 4.1% de que...
Hay AL MENOS entre 5 y 5 "manzanas podridas" sentadas en la bancada de PP del congreso
Hay AL MENOS entre 3 y 3 "manzanas podridas" sentadas en la bancada de PSOE del congreso
Hay 

Ya se ve que las probabilidades de tener sentado un corrupto en el congreso, para cada partido, crecerían muy rápidamente. **90.5% para el PP**, 59.3% para el PSOE, 4.1% para Podemos, y 3.5% para Ciudadanos.